# Iteratorパターン

* Iteratorパターンとは
    * 何かがたくさん集まっているときに、それを順番に指し示していき全体をスキャンしていく処理を行うためのもの
    * iterate
        * 何かを繰り返すという意味の単語
        * 日本語では反復子と呼ばれることがある
        * 種類
            * 順方向に一度だけスキャンする (オーソドックス)
            * 最後尾から開始して逆方向に進む
            * 順方向にも逆方向にも進む (nextメソッドだけではなくpreviousメソッドも持つ)
            * 番号を指定していきなり底にジャンプする
        * deleteIteratorは不要
            * 言語によっては自動的にインスタンスが削除されないがJavaは削除されるので不要
    * 数え上げの仕組みがAggregate役の外に置かれている
        * 1つのConcreteAggregate役に対して複数のConcreteIterator役を作ることができる

## Aggregateインタフェース

* Aggregateインタフェース
    * 数え上げを行うものの<font color="red">集合体</font>を表す
    * このインタフェースを実装しているクラスは、配列のように<font color="red">何かがたくさん集まっているも</font>のになる
    * aggregate
        * 集める、集まる、集合という意味の英単語
    * 宣言されているメソッド
        * iteratorメソッド
            * 集合に対応するIteratorを1個作成するためのもの
            * 使う状況
                * 集合体を数え上げたい
                * 集合体をスキャンしたい
                * 集合体を1つずつ調べていきたい
            * このメソッドを使ってIteratorインタフェースを実装したクラスのインスタンスを1個作る

In [9]:
%%bash
### 変数の設定
name=Aggregate

### ソースの編集
cat <<- EOS > ${name}.java
public interface ${name} {
    public abstract Iterator iterator();
}
EOS

### コンパイル
javac ${name}.java
#javac -encoding UTF-8 ${name}.java

## Iteratorインタフェース

* Iteratorインタフェース
    * <font color="red">要素の数え上げ</font>を行うもの
        * ループ変数のような役割を果たすもの
    * 宣言されているメソッド
        * hasNextメソッド
            * 次の要素が存在するかどうかを調べる
                * <font color="red">次にnextメソッドを呼んでも大丈夫かどうかを調べる</font>
            * ループの終了条件で使うためのもの
            * 戻り値はboolean型
                * 有ればtrue、無ければfalse
        * nextメソッド
            * 次の要素を得る
                * 戻り値はObject型
                    * 集合体の要素を一個返す
            * 次の要素を返すように内部状態を次に進めておくという隠れ仕事もある
                * 隠れ仕事はIteratorインタフェースを実装しているクラスの方に具体的な動きが記載されている
                    * ここではBookShelfIteratorクラス
            * <font color="red">現在の要素を返しつつ次の位置に進める</font>

In [8]:
%%bash
### 変数の設定
name=Iterator

### ソースの編集
cat <<- EOS > ${name}.java
public interface ${name} {
    public abstract boolean hasNext();
    public abstract Object next();
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

## Bookクラス

* 本を表すクラス
    * 宣言されているメソッド
        * getNameメソッド
            * 本の名前を得る
            * 本の名前はコンストラクタでインスタンスを初期化するときに引数で指定する

In [7]:
%%bash
### 変数の設定
name=Book

### ソースの編集
cat <<- EOS > ${name}.java
public class ${name} {
    private String name;
    public Book(String name) {
        this.name = name;
    }
    public String getName() {
        return name;
    }
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

## BookShelfクラス

* 本棚を表しているクラス
    * Aggregateインタフェースを実装している
        * 「implements Aggregate」がAggregateインタフェースを実装していることを表現している
        * Aggregateインタフェースで宣言されていたiteratorメソッドの実態が書かれている

In [10]:
%%bash
### 変数の設定
name=BookShelf

### ソースの編集
cat <<- EOS > ${name}.java
public class ${name} implements Aggregate {
    //# Bookの配列を持つbooksフィールド。privateにして外から不用意に変更されることを防ぐ
    private Book[] books;
    private int last = 0;
    
    public BookShelf(int maxsize) {
    　　　　//# maxsizeは配列の大きさで、最初にBookShelfのインスタンスを作るときに指定する
        this.books = new Book[maxsize];
    }
    
    public Book getBookAt(int index) {
        return books[index];
    }
    
    public void appendBook(Book book) {
        this.books[last] = book;
        last++;
    }
    
    public int getLength() {
        return last;
    }
    
    //# BookShelfクラスに対応するIteratorとしてBookShelfIteratorというクラスのインスタンスを生成してそれを返すメソッド
    //# 本棚の本を数えたいときに呼び出す
    public Iterator iterator() {
        return new BookShelfIterator(this);
    }
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

## BookShelfIteratorクラス

* BookShelfクラスのスキャンを行うクラス
    * Iteratorインタフェースを実装している
        * BookShelfIteratorをIteratorとして扱うため

In [11]:
%%bash
### 変数の設定
name=BookShelfIterator

### ソースの編集
cat <<- EOS > ${name}.java
public class ${name} implements Iterator {
    //# BookShelfIteratorがスキャンする本棚に相当するbookshelfフィールド
    private BookShelf bookShelf;
    //# 現在注目している本を指す添字に相当するindexフィールド
    private int index;
    
    //# 渡されたBookShelfインスタンスをbookShelfフィールドに保存し、indexを0にしているコンストラクタ
    public BookShelfIterator(BookShelf bookShelf) {
        this.bookShelf = bookShelf;
        this.index = 0;
    }
    
    //# Iteratorインタフェースで宣言されているメソッドを実装したhasNextメソッド
    //# indexが本棚の本の冊数より小さいかどうかで判断している
    public boolean hasNext() {
        if (index < bookShelf.getLength()) {
            return true;
        } else {
            return false;
        }
    }
    
    //# Iteratorインタフェースで宣言されているメソッドを実装したnextメソッド
    //# 現在注目している本(Bookのインスタンス)を返し、更に次に進める
    public Object next() {
        //# 戻り値として返すべき本をbookという変数にとっておく
        Book book = bookShelf.getBookAt(index);
        //# indexを次に進める
        index++;
        return book;
    }
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

## Mainクラス

In [31]:
%%bash
### 変数の設定
name=Main

### ソースの編集
cat <<- EOS > ${name}.java
public class ${name} {
    public static void main(String[] args) {
        //# 本が4冊入る本棚を作る
        BookShelf bookShelf = new BookShelf(4);
        
        //# 4冊の本を作って入れる
        bookShelf.appendBook(new Book("Around the World in 80 Days"));
        bookShelf.appendBook(new Book("Bible"));
        bookShelf.appendBook(new Book("Cinderella"));
        bookShelf.appendBook(new Book("Daddy-Long-Legs"));
        
        //# 本棚をスキャンするためのIteratorのインスタンスitを生成
        Iterator it = bookShelf.iterator();
        
        //# 本がある限りwhileループが回る
        while (it.hasNext()) {
            //# 本を1冊1冊調べる
            Book book = (Book) it.next();
            System.out.println(book.getName());
        }
    }
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

### 実行
#java -classpath ${classpath} ${name}
java ${name}

Around the World in 80 Days
Bible
Cinderella
Daddy-Long-Legs


In [24]:
%%bash

current_dir=`pwd`
isbn="ISBN4-7973-2703-0"
fig="Fig.1-2"

### サンプルコードの作成
cat <<- EOS > ${current_dir}/plantuml/${isbn}_${fig}.pu
@startuml
skinparam classAttributeIconSize 0

interface Aggregate {
    {abstract} +iterator()
}

interface Iterator {
    {abstract} +hasNext()
    {abstract} +next()    
}

class Book {
    -name : String
    +getName()
}

class BookShelf {
    -books : Book[]
    -last : int
    +getBookAt()
    +appendBook()
    +getLength()
    +iterator()
}

class BookShelfIterator {
    -bookShelf : BookShelf
    -index : int
    +hasNext()
    +next()
}


Aggregate --> Iterator : Creates >
Aggregate <|.. BookShelf
Iterator <|.. BookShelfIterator
BookShelf <--o BookShelfIterator
Book <--o BookShelf

@enduml
EOS

### サンプルコードからUMLの図を作成
java -jar plantuml.jar -o ${current_dir}/img -tpng ${current_dir}/plantuml/${isbn}_${fig}.pu

<img src="img/ISBN4-7973-2703-0_Fig.1-2.png" alt="サンプルプログラムのクラス図" title="サンプルプログラムのクラス図" align="left" />  
<br clear="left">

* Bookクラス
    * getNameメソッドで本の名前を得る
    * 本の名前はコンストラクタでインスタンス生成時に引数で指定する
* Iteratorインタフェース
    * 要素の数え上げをするためのメソッドを宣言している
        * 次の要素が存在するか調べるhasNextメソッド
        * 次の要素を得るためのnextメソッド
* BookShelfIteratorクラス
    * Iteratorインタフェースを実装しているクラス
        * hasNext, nextメソッドの実体がある
* Aggregateインタフェース
    * Iteratorを生成するメソッドが宣言されている
        * 集合体に対応するIteratorを1個生成するためのiteratorメソッド
* BookShelfクラス
    * Aggregateインタフェースを実装しているクラス
        * Aggregateインタフェースで宣言されていた<font color="red">iteratorメソッドの実体</font>がある
            * BookShelfクラスに対応するIteratorとしてBookShelfIteratorというクラスのインスタンスを生成して返す
    * Bookの配列を持つ
        * BookShelfクラスのインスタンスはBookクラスのインスタンスを複数持つ

In [ ]:
%%bash

current_dir=`pwd`
isbn="ISBN4-7973-2703-0"
fig="Fig.1-4"

### サンプルコードの作成
cat <<- EOS > ${current_dir}/plantuml/${isbn}_${fig}.pu
@startuml
skinparam classAttributeIconSize 0

interface Aggregate {
    {abstract} +iterator()
}

interface Iterator {
    {abstract} +hasNext()
    {abstract} +next()    
}

class ConcreteAggregate {
    +iterator()
}

class ConcreteIterator {
    -aggregate
    +hasNext()
    +next()
}


Aggregate --> Iterator : Creates >
Aggregate <|.. ConcreteAggregate
Iterator <|.. ConcreteIterator
ConcreteAggregate <--o ConcreteIterator

@enduml
EOS

### サンプルコードからUMLの図を作成
java -jar plantuml.jar -o ${current_dir}/img -tpng ${current_dir}/plantuml/${isbn}_${fig}.pu

## Iteratorパターンの登場人物

<img src="img/ISBN4-7973-2703-0_Fig.1-4.png" alt="Iteratorパターンのクラス図" title="Iteratorパターンのクラス図" align="left" />  
<br clear="left">

* Iterator(反復子)役
    * 要素を順番にスキャンしていく<font color="red">インタフェース(API)</font>を定める役
        * サンプルプログラムではInterfaceインタフェースがこの役をつとめる
            * hasNextメソッドとnextメソッドを定める
* ConcreteIterator(具体的な反復子)役
    * Iterator役が定めたインタフェース(API)を実際に実装する役
        * サンプルプログラムではBookShelfIteratorクラスがこの役をつとめる
    * スキャンするために必要な情報を持っている必要がある
        * サンプルプログラム
            * BookShelfクラスのインスタンスをbookShelfフィールドで覚えている
            * 注目している本をindexフィールドで覚えている
            * hasNextメソッドとnextメソッドを実装している
* Aggregate(集合体)役
    * Iterator役を作り出すインタフェース(API)を定める役
        * サンプルプログラムではAggregateインタフェースがこの役をつとめる
            * iteratorメソッドを定めている
                * 「私が持っている要素を順番にスキャンしてくれる人」を作り出すメソッド
* ConcreteAggregate(具体的な集合体)役
    * Aggregate役が定めたインタフェース(API)を実際に実装する役
        * サンプルプログラムではBookShelfクラスがこの役をつとめる
            * iteratorメソッドを実装している
            

In [36]:
%%bash
# 以下練習問題用

# =================== #
name=Aggregate
cat <<- EOS > ${name}.java

public interface ${name} {
    public abstract Iterator iterator();
}

EOS

# =================== #
name=Iterator
cat <<- EOS > ${name}.java

public interface ${name} {
    public abstract boolean hasNext();
    public abstract Object next();
}

EOS

# =================== #
name=Book
cat <<- EOS > ${name}.java

public class ${name} {
    private String name;
    public Book(String name) {
        this.name = name;
    }
    public String getName() {
        return name;
    }
}

EOS

# =================== #
name=BookShelf
cat <<- EOS > ${name}.java

public class ${name} implements Aggregate {
    private Book[] books;
    private int last = 0;
    
    public BookShelf(int maxsize) {
        this.books = new Book[maxsize];
    }
    
    public Book getBookAt(int index) {
        return books[index];
    }
    
    public void appendBook(Book book) {
        this.books[last] = book;
        last++;
    }
    
    public int getLength() {
        return last;
    }
    
    public Iterator iterator() {
        return new BookShelfIterator(this);
    }
}

EOS

# =================== #
name=BookShelfIterator
cat <<- EOS > ${name}.java

public class ${name} implements Iterator {
    private BookShelf bookShelf;
    private int index;
    
    public BookShelfIterator(BookShelf bookShelf) {
        this.bookShelf = bookShelf;
        this.index = 0;
    }
    
    public boolean hasNext() {
        if (index < bookShelf.getLength()) {
            return true;
        } else {
            return false;
        }
    }
    
    public Object next() {
        Book book = bookShelf.getBookAt(index);
        index++;
        return book;
    }
}

EOS

# =================== #
name=Main
cat <<- EOS > ${name}.java

public class ${name} {
    public static void main(String[] args) {
        BookShelf bookShelf = new BookShelf(4);
        
        bookShelf.appendBook(new Book("Around the World in 80 Days"));
        bookShelf.appendBook(new Book("Bible"));
        bookShelf.appendBook(new Book("Cinderella"));
        bookShelf.appendBook(new Book("Daddy-Long-Legs"));
        
        //# Iterator型の変数にbookShelf.iteratorの返り値を代入している(BookShelfIterator型ではない)
        //# あくまでIteratorのメソッドを使ってプログラミング行おうという姿勢を示している
        Iterator it = bookShelf.iterator();
        
        while (it.hasNext()) {
            Book book = (Book) it.next();
            System.out.println(book.getName());
        }
    }
}

EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

### 実行
#java -classpath ${classpath} ${name}
java ${name}

Around the World in 80 Days
Bible
Cinderella
Daddy-Long-Legs
